In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np
import requests
import time
import re

In [2]:
df = pd.read_csv('kakaoAPI_data.csv', encoding='utf-8')

In [3]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df.head()

,ID,stores,X,Y,road_address,place_url
0,20908598,공차 영등포구청점,126.897061,37.526699,서울 영등포구 당산로 136,http://place.map.kakao.com/20908598
1,1714644708,이도숙성회,126.896237,37.526695,서울 영등포구 당산로31길 4,http://place.map.kakao.com/1714644708
2,27527233,까치네분식,126.896689,37.527085,서울 영등포구 당산로 139,http://place.map.kakao.com/27527233
3,22246273,푸른농장 해바라기생고기정육식당 영등포점,126.896155,37.527045,서울 영등포구 당산로33길 5,http://place.map.kakao.com/22246273
4,19609040,사조참치 영등포구청점,126.896554,37.526706,서울 영등포구 당산로 133,http://place.map.kakao.com/19609040


In [12]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

/var/folders/6b/s1063_qj51s2xh9qm5l7ntq40000gn/T/ipykernel_1026/3272586871.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [13]:
# 수집할 정보
menu = []
category = []
business_hours = []
rating = []
phone_number = [] 
last_update = []
delay = 0.5

p = re.compile('[0-9]{2}')

In [16]:
for url in tqdm(df['place_url']):
    driver.get(url)
    time.sleep(1.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
# 기본 정보와 메뉴가 구역으로 분리되어 있어 각각 변수를 만들어 진행
    basic = soup.find('div', 'cont_essential') # 기본 정보 모음
################################################ 기본 정보 수집 ################################################
    try:
        category.append(basic.find('span', 'txt_location').text.split(':')[1].strip()) # 분류: 회
    except :
        category.append(np.NaN)
    try:
        business_hours.append(basic.find('span', 'txt_operation').text[:-1]) # '월~토 11:00 ~ 22:00\n'
    except :
        business_hours.append(np.NaN)
    try:
        rating.append(basic.find('span', 'color_b').text[:-1]) # 3.9점
    except :
        rating.append(np.NaN)
    try:
        phone_number.append(basic.find('span', 'txt_contact').text)
    except :
        phone_number.append(np.NaN)
    try:
        last_update.append(basic.find('span', 'date_revise').text)
    except :
        last_update.append(np.NaN)
    
    time.sleep(delay)
    
    ################################################ 메뉴 정보 수집 ################################################
    try:
        cont_menu = soup.find('div', 'cont_menu') # 메뉴 정보 모음
        menues = cont_menu.find_all('li')
        menu_temp = []
        for x in menues:
            menu_temp.append(x.find('span', 'loss_word').text)
        menu.append(menu_temp)
    except :
        menu.append(np.NaN)    
    time.sleep(delay)

100%|██████████| 1412/1412 [1:06:46<00:00,  2.84s/it]


In [16]:
# driver.close()

In [17]:
print(len(category), len(business_hours), len(rating), len(phone_number), len(last_update), len(menu))

1417 1417 1417 1417 1417 1417


In [ ]:
df['category'] = category
df['business_hours'] = business_hours
df['rating'] = rating
df['phone_number'] = phone_number
df['last_update'] = last_update
df['menu'] = menu
df.to_csv('kakaomapAPI_crawling(selenium)', index=False, encoding='utf-8-sig')